In [3]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
SERPER_API_KEY = os.getenv("SERPER_API_KEY")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
LANGSMITH_PROJECT = os.getenv("LANGSMITH_PROJECT")

In [5]:
os.environ["TAVILY_API_KEY"]= TAVILY_API_KEY
os.environ["LANGCHAIN_API_KEY"]= LANGCHAIN_API_KEY
os.environ["GROQ_API_KEY"]= GROQ_API_KEY
os.environ["SERPER_API_KEY"]= SERPER_API_KEY
os.environ["GOOGLE_API_KEY"]= GOOGLE_API_KEY
os.environ["LANGSMITH_TRACING"]= "true"
os.environ["LANGSMITH_ENDPOINT"]= "https://api.smith.langchain.com"
os.environ["LANGSMITH_PROJECT"]= LANGSMITH_PROJECT

In [6]:
LANGSMITH_PROJECT

'pre-req'

In [7]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

d:\langgraph\lang-env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
while True:
    question = input("Ask your question: ")
    if question!="quit":
        print(llm.invoke(question).content)
    else:
        print("Have a great day. Bye!")
        break

Have a great day. Bye!


## Integrating memory

In [9]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import AIMessage

In [10]:
store ={}

In [11]:
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

In [12]:
config = {"configurable": {"session_id":"first_chat"}}

In [13]:
model_with_memory = RunnableWithMessageHistory(llm,get_session_history)

In [14]:
model_with_memory.invoke(("Hello. I am Shochcho"), config=config).content

"Hello Shochcho!  It's nice to meet you. How can I help you today?"

In [15]:
model_with_memory.invoke(("What is my name?"), config=config).content

'Your name is Shochcho.'

In [16]:
model_with_memory.invoke(("Do you know my age?"), config=config).content

'No, I do not know your age.  I only know what you have told me, which is your name.'

In [17]:
model_with_memory.invoke(("I am 24 years old."), config=config).content

'Okay, Shochcho.  I now know you are 24 years old.  Is there anything else I can help you with?'

In [18]:
model_with_memory.invoke(("Now, do you know my age?"), config=config).content

'Yes, I know you are 24 years old.'

In [19]:
store

{'first_chat': InMemoryChatMessageHistory(messages=[HumanMessage(content='Hello. I am Shochcho', additional_kwargs={}, response_metadata={}), AIMessage(content="Hello Shochcho!  It's nice to meet you. How can I help you today?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-076ced2d-f185-49c6-bf39-47179defd6e8-0', usage_metadata={'input_tokens': 8, 'output_tokens': 22, 'total_tokens': 30, 'input_token_details': {'cache_read': 0}}), HumanMessage(content='What is my name?', additional_kwargs={}, response_metadata={}), AIMessage(content='Your name is Shochcho.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-a2e79aed-9111-4890-b856-78dc7326554d-0', usage_metadata={'input_tokens': 36, 'output_tokens': 8, 'total_tokens': 44, 'input_token_details': {'cache_read': 0}}), H

## RAG

In [21]:
from langchain_community.document_loaders import TextLoader, DirectoryLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain import PromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough , RunnableLambda
from langchain_core.output_parsers import StrOutputParser

In [23]:
# Step 1: Reading the txt files from source directory
loader = DirectoryLoader('../data', glob="./*.pdf", loader_cls=PyPDFLoader)
docs = loader.load()

# Step 2: Creating Chunks using RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len
)
new_docs = text_splitter.split_documents(documents=docs)
doc_strings = [doc.page_content for doc in new_docs]

# Step 3: Creating Retriever using FAISS Vector DB

# Create the FAISS database from the document embeddings
db = FAISS.from_documents(new_docs, embeddings)

# Create a retriever from the FAISS database
retriever = db.as_retriever(search_kwargs={"k": 4})

# Example usage of retriever
query = "What are the Basics of Python Programming?"
results = retriever.get_relevant_documents(query)
for idx, result in enumerate(results, 1):
    print(f"Result {idx}:\n{result.page_content}\n")

C:\Users\User\AppData\Local\Temp\ipykernel_17468\2339167315.py:24: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  results = retriever.get_relevant_documents(query)


Result 1:
Introduction  19
 https://doi.org/10.15215/remix/9781998944088.01
In Python, however, you only need to write the following code and save to 
a file, say, myhelloworld .py:
print("Hello World!")
Then run the following command:
python myhelloworld .py
More importantly, Python supports different programming paradigms, 
including structured programming, imperative programming, object- oriented 
programming, functional programming, and procedural programming.  
Logic programming is probably the only exception; it is not supported by Python.
Python is very powerful for two reasons. The first reason, as you will see 
later, is that Python has many powerful language constructs and statements to 
represent data and operations on various data. The second reason is that there 
are thousands of third- party packages/libraries/modules available for all kinds 
of programming needs in addition to the large standard libraries included

Result 2:
https://doi.org/10.15215/remix/9781998944088.0

In [24]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = PromptTemplate.from_template(template)

## LCEL

In [25]:
retrieval_chain = (
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
    | prompt
    | llm
    | StrOutputParser()
    )

In [26]:
question ="What is for loop? And how can I print star pattern using loop?"
print(retrieval_chain.invoke(question))

Based on the provided text, a for loop in Python is one of two statements used to create loops or iterations.  It takes the form:  `for <iteration variable> in <sequence to be looped through>:` followed by a code block. The iteration variable takes items one by one from the sequence (which can be a list, tuple, string, or iterable object).  The example provided does not show how to print a star pattern.


## Tools and Agents

In [27]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [28]:
api_wrapper = WikipediaAPIWrapper(top_k_results=5, doc_content_chars_max=100)

In [29]:
tool = WikipediaQueryRun(api_wrapper=api_wrapper)

In [30]:
tool.description

'A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.'

In [31]:
tool.args

{'query': {'description': 'query to look up on wikipedia',
  'title': 'Query',
  'type': 'string'}}

In [32]:
print(tool.run({"query": "Gemini"}))

d:\langgraph\lang-env\Lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file d:\langgraph\lang-env\Lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


Page: Gemini (astrology)
Summary: Gemini (♊︎) ( JEM-in-eye  Greek: Δίδυμοι, romanized: Dídymoi, Lati


In [33]:
from langchain_community.tools.tavily_search import TavilySearchResults

In [34]:
tool = TavilySearchResults()

In [35]:
tool.invoke({"query": "What happened in the latest burning man floods"})

[{'url': 'https://abcnews.go.com/US/burning-man-flooding-happened-stranded-festivalgoers/story?id=102908331',
  'content': 'Tens of thousands of Burning Man attendees are now able to leave the festival after a downpour and massive flooding left them stranded over the weekend.. The festival, held in the Black Rock'},
 {'url': 'https://www.nbcnews.com/news/us-news/live-blog/live-updates-burning-man-flooding-keeps-thousands-stranded-nevada-site-rcna103193',
  'content': "Profile\nSections\ntv\nFeatured\nMore From NBC\nFollow NBC News\nnews Alerts\nThere are no new alerts at this time\nBurning Man flooding keeps thousands stranded at Nevada site as authorities investigate 1 death\nBurning Man attendees struggling to get home\n70,000+ stuck at Burning Man: When will they be able to get out?\n Thousands still stranded at Burning Man after torrential rain\nBurning Man revelers unfazed by deluge and deep mud\nReuters\nThousands of Burning Man attendees partied hard on Sunday despite downpours 

In [36]:
from langchain.agents import AgentType
from langchain.agents import load_tools 
from langchain.agents import initialize_agent

In [37]:
tool=load_tools(["wikipedia"],llm=llm)

In [38]:
agent=initialize_agent(tool,llm,agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,verbose=True)

C:\Users\User\AppData\Local\Temp\ipykernel_17468\2645007812.py:1: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent=initialize_agent(tool,llm,agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,verbose=True)


In [39]:
agent.run("can you tell me what is current GDP of Bangladesh?")

C:\Users\User\AppData\Local\Temp\ipykernel_17468\1204910422.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  agent.run("can you tell me what is current GDP of Bangladesh?")




> Entering new AgentExecutor chain...
Thought: I need to find the current GDP of Bangladesh.  Wikipedia is a good source for this type of information.

Action: wikipedia
Action Input: GDP of Bangladesh
Observation: Page: Economy of Bangladesh
Summary: The economy of Bangladesh is a major developing mixed economy. As the second-largest economy in South Asia, Bangladesh's economy is the 35th largest in the world in nominal terms, and 25th largest by purchasing power parity. Bangladesh is seen by various financial institutions as one of the Next Eleven. It has been transitioning from being a frontier market into an emerging market. Bangladesh is a member of the South Asian Free Trade Area and the World Trade Organization. In fiscal year 2021–2022, Bangladesh registered a GDP growth rate of 7.2% after the global pandemic. Bangladesh is one of the fastest growing economies in the world.
Industrialisation in Bangladesh received a strong impetus after the partition of India due to labour re

"I cannot provide the exact current GDP of Bangladesh.  Wikipedia articles on Bangladesh's economy and lists of countries by GDP refer to the IMF and World Bank as sources for this type of data, but do not provide the precise current figure.  To obtain the most up-to-date and precise figure, you would need to consult the IMF's World Economic Outlook database directly."